In [92]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import random
from sklearn.linear_model import SGDClassifier as LC
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier as AB
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as NB
from nltk.corpus import stopwords
from sklearn.ensemble import ExtraTreesClassifier as ET
from os.path import dirname, abspath, join
import praw
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from  sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.neural_network import MLPClassifier as MLPC
from sklearn.neural_network import MLPRegressor as MLPR
from nltk.stem import PorterStemmer

import re
from sklearn.decomposition import PCA
from sklearn import preprocessing
import datetime

In [93]:
d = dirname(abspath(''))
train_dic = join(d, 'dataset/Phase 2 - stockDirection/train.csv')
test_dic = join(d, 'dataset/Phase 2 - stockDirection/test.csv')
c_dic = join(d, 'dataset/Phase 2 - stockDirection/combined_stock_data.csv')
sub_dic = join(d,  'submission/Phase 2 - submission/submission_phase2_yourteamname.csv' )
write_dic = join(d, 'submission/Phase 2 - submission/submission_phase2_team1.csv')

In [94]:
# load all data
raw_training = pd.DataFrame.from_csv(train_dic)
raw_test = pd.DataFrame.from_csv(test_dic)
sid = SentimentIntensityAnalyzer()
cdata = pd.DataFrame.from_csv(c_dic)
sub_data = pd.DataFrame.from_csv(sub_dic, index_col=None)
sub_data.reset_index(inplace = True)
cdata['Date']= [datetime.datetime.strptime(cdata['Date'][i], "%Y-%m-%d")for i in range(len(cdata))]
raw_training.sort_index(inplace = True)
raw_training.reset_index(inplace = True)
raw_test.reset_index(inplace = True)
raw_test.sort_index(inplace = True)

In [95]:
list(raw_training['Close'] - raw_training['Open'] > 0).count(True)/len(raw_training['Close'] - raw_training['Open'] > 0)

0.5380263984915148

In [96]:
def test_acc(test):
    p_i, cor, icor = 0, 0, 0
    for c_i in range(len(cdata)):
        if p_i == len(test):
            break
        if cdata['Date'][c_i] == test['Date'][p_i]:
            if cdata['Label'][c_i] == test['prediction'][p_i]:
                cor += 1
            else:
                icor += 1
            p_i += 1
    print(cor/(cor+icor))

In [97]:
def write_data(sub_data, test):
    for i in range(len(sub_data)):
        d = datetime.datetime.strptime(sub_data['Date'][i], "%m/%d/%y")
        sub_data['stock_directionality'][i]  = int(test['prediction'][test['Date'] == d].tolist()[0])


    sub_data.drop('index', axis= 1, inplace = True)
    sub_data.to_csv(write_dic, index = False)

In [98]:
#give predictrion
tr_next,tr_before, te_i = 1, 0, 0
before_date, before_close = raw_training['Date'][tr_before], raw_training['Close'][tr_before]
nxt_date, nxt_open = raw_training['Date'][tr_next], raw_training['Open'][tr_next]
prediction = []
while te_i < len(raw_test):
    te_date = raw_test['Date'][te_i]
    while nxt_date < te_date:
        tr_next += 1
        nxt_date, nxt_open = raw_training['Date'][tr_next], raw_training['Open'][tr_next]
    while raw_training['Date'][tr_before+1] < te_date:
        tr_before += 1
        before_date, before_close = raw_training['Date'][tr_before], raw_training['Close'][tr_before]
    if nxt_open > before_close:
        prediction.append(1)
    else:
        prediction.append(0)
    te_i += 1
raw_test['prediction'] = prediction

In [99]:
cor, incor = 0, 0
for i in range(1, len(raw_training)-1):
    dif = max(raw_training['Date'][i+1] - raw_training['Date'][i], raw_training['Date'][i] - raw_training['Date'][i-1])
    if dif > datetime.timedelta(days = 1):
        pre_close, after_open, real = raw_training['Close'][i-1], raw_training['Open'][i+1], raw_training['Close'][i] - raw_training['Open'][i]
        if (after_open - pre_close )* real > 0:
            cor += 1
        else:
            incor += 1
print(cor/(cor+incor))
print(cor, incor)   

0.832146490335707
818 165


In [100]:
def accuracy(model, test_x, test_y):
    pre = model.predict(test_x)
    res = [pre[i]== test_y[i] for i in range(len(test_y))]
    return 1.0* res.count(True)/len(res)

In [101]:
dis = abs(raw_training['Close'] - raw_training['Open'])
dis.describe()

count    1591.000000
mean       97.666400
std        98.530411
min         0.000000
25%        27.880375
50%        68.000000
75%       134.524905
max       886.399902
dtype: float64

In [102]:
def imp_words(train_index, raw_training):
    word_dic = {}
    stop_words = set(stopwords.words('english'))
    regex = re.compile('[^a-z]') 
    ps = PorterStemmer()

    for i in train_index:
        for j in range(1, 26):
            if type(raw_training['Top'+str(j)][i]) != str:
                continue

            news = raw_training['Top'+str(j)][i][2:-1]
            news = news.lower()

            news = regex.sub(' ', news) 
            news = ' '.join(news.split())
            news = news.replace('u s', 'us')
            news = news.replace('u k', 'uk')
            news = news.replace('u n', 'un')
            news = news.replace('n korea', 'north korea')
            news = news.replace('e mail', 'email')
            for w in news.split():

                if w in stop_words or ps.stem(w) in stop_words:
                    continue
                    
                word_dic[ps.stem(w)] = word_dic.get(ps.stem(w), 0) + 1
    
    keywords = {key:index for index, key in enumerate(word_dic.keys()) if word_dic[key] > 50}
    influence = {}
    for i in train_index:
        for j in range(1, 26):
            if type(raw_training['Top'+str(j)][i]) != str:
                continue

            news = raw_training['Top'+str(j)][i][2:-1]
            news = news.lower()

            news = regex.sub(' ', news) 
            news = ' '.join(news.split())
            news = news.replace('u s', 'us')
            news = news.replace('u k', 'uk')
            news = news.replace('u n', 'un')
            news = news.replace('n korea', 'north korea')
            news = news.replace('e mail', 'email')
            for w in news.split():
                stem = ps.stem(w)
                if stem in keywords:
                    influence[stem] = influence.get(stem, 0) +  raw_training['Close'][i] - raw_training['Open'][i]
    avg_dif = {}
    for key in influence:
        avg_dif[key] = abs(influence[key]/word_dic[key])
    return avg_dif
    
    



In [184]:
def generate_training_set(df, sid, train_set, valid_set, last_days):
    train_x, valid_x, train_y, valid_y  = [], [], [], []
    sentiment_keys = ['compound', 'neg', 'pos']
    """
    avg_dif_dic = imp_words(train_set, raw_training)
    i = 0
    imp_words_dic = {}
    stop_words = set(stopwords.words('english'))
    regex = re.compile('[^a-z]') 
    ps = PorterStemmer()
    
    for key in avg_dif_dic:
        #avg influence at least be 50
        if avg_dif_dic[key]>30:
            imp_words_dic[key] =  i
            i += 1
    """
    
    for row_index in range(len(df)):
        increase = None
        # ylabel
        if df['Close'][row_index] - df['Open'][row_index] >= 0:
            increase = 1
        else:
            increase = 0
        d = []
        #imp_list= [0]*len(imp_words_dic.keys())
        for j in range(1, 26):
            #sentiment score
            polarity_scores = {'neu':0, 'pos':0, 'neg':0,'compound':0}
            try:
                polarity_scores = sid.polarity_scores(df['Top' + str(j)][row_index])
            except:
                pass
            for key in sentiment_keys:
                d.append(polarity_scores[key])
            #important words
            """
            if type(df['Top'+str(j)][row_index]) != str:
                continue

            news = raw_training['Top'+str(j)][row_index][2:-1]
            news = news.lower()

            news = regex.sub(' ', news) 
            news = ' '.join(news.split())
            news = news.replace('u s', 'us')
            news = news.replace('u k', 'uk')
            news = news.replace('u n', 'un')
            news = news.replace('n korea', 'north korea')
            news = news.replace('e mail', 'email')
            for w in news.split():
                stem = ps.stem(w)
                if stem in imp_words_dic:
                    imp_list[imp_words_dic[stem]] += 1
            
        d += imp_list
        """
        last = pd.Series([None] * last_days)
        # last n days data
        try:
            for i in range(last_days):
                last[i] = raw_training['Close'][row_index-1-i] - raw_training['Open'][row_index-1-i]
        except:
            pass
        d += list(last)
        
        if row_index in train_set:
            train_x.append(d)
            train_y.append(increase)
        else:
            valid_x.append(d)
            valid_y.append(increase)
        
    column_names = []
    for i in range(1, 26):
        for key in sentiment_keys:
            column_names.append('Top'+str(i)+'_'+key)
    #column_names += list(imp_words_dic.keys())
    column_names += ['last'+str(i+1) for i in range(last_days)]
    train_df = pd.DataFrame(data = train_x, columns = column_names)
    valid_df = pd.DataFrame(data = valid_x, columns = column_names)
    return train_df, train_y, valid_df, valid_y
        

In [216]:
def generate_test_set(df, sid, last_days):
    test_x  = []
    sentiment_keys = ['compound', 'neg', 'pos']
    for row_index in range(len(df)): 
        d = []
        for j in range(1, 26):
            #sentiment score
            polarity_scores = {'neu':0, 'pos':0, 'neg':0,'compound':0}
            try:
                polarity_scores = sid.polarity_scores(df['Top' + str(j)][row_index])
            except:
                pass
            for key in sentiment_keys:
                d.append(polarity_scores[key])
        last = pd.Series([None] * last_days)
        d += list(last)
        test_x.append(d)
    column_names = []
    for i in range(1, 26):
        for key in sentiment_keys:
            column_names.append('Top'+str(i)+'_'+key)
    column_names += ['last'+str(i+1) for i in range(last_days)]
    test_df = pd.DataFrame(data = test_x, columns = column_names)
    return test_df
        

In [106]:
train_ratio = 0.8
index_list= [i for i in range(len(raw_training))]
random.shuffle(index_list)
sid = SentimentIntensityAnalyzer()
last_days = 0
train_set, test_set = set(index_list[:int(train_ratio*len(raw_training))]), set(index_list[int(train_ratio*len(raw_training)):])
train_x, train_y, valid_x, valid_y = generate_training_set(raw_training, sid, train_set, test_set, last_days)
for n in train_x.columns:
    train_x[n].fillna(train_x[n].mean(), inplace = True)
    valid_x[n].fillna(train_x[n].mean(), inplace = True)
sum([1 for i in range(len(valid_y)) if valid_y[i] > 0 ])/len(valid_y)

0.5172413793103449

In [107]:
train_x.head()

,Top1_compound,Top2_compound,Top3_compound,Top4_compound,Top5_compound,Top6_compound,Top7_compound,Top8_compound,Top9_compound,Top10_compound,...,Top16_compound,Top17_compound,Top18_compound,Top19_compound,Top20_compound,Top21_compound,Top22_compound,Top23_compound,Top24_compound,Top25_compound
0,-0.5994,0.0000,-0.3612,-0.7089,-0.9260,0.0000,-0.2732,0.2144,-0.5719,-0.5994,...,0.0000,0.0000,0.2086,0.2023,0.0258,-0.7579,-0.6249,-0.2755,-0.8519,0.4019
1,0.8156,-0.3182,0.4404,-0.1965,0.0000,-0.4939,-0.5106,-0.0772,-0.6688,-0.3400,...,0.0000,0.0000,-0.6597,0.0000,0.0000,-0.8020,0.0000,-0.3182,-0.1832,0.0000
2,-0.7184,-0.8074,-0.6369,-0.1280,-0.5106,0.0000,0.2960,0.0000,0.3612,0.0000,...,0.0000,0.0000,0.0000,-0.5423,-0.0258,-0.2960,0.4939,-0.5719,-0.4215,-0.3400
3,0.2023,0.0000,0.6808,-0.8689,-0.6124,-0.6369,0.7177,-0.4404,-0.6808,-0.3400,...,-0.6249,0.0000,0.0000,0.0000,0.0000,-0.4404,-0.5994,0.1779,-0.6908,0.7096
4,-0.2732,-0.4767,-0.8316,0.0000,0.0000,0.4588,0.0000,-0.1027,0.0000,-0.2732,...,0.0000,-0.2732,0.0000,0.0000,-0.5994,-0.2960,-0.2500,-0.3612,0.2500,0.0000


In [108]:
len(train_x.columns)

25

In [111]:
mlp = MLPC(activation = 'relu', solver = 'adam', alpha = 0.1, hidden_layer_sizes = tuple((200,)*3), tol = 0.0001,max_iter= 200000, verbose =False, learning_rate = 'adaptive' )
mlp.fit(train_x, train_y)
print(accuracy(mlp, valid_x, valid_y))

0.5235109717868338


ANN

In [153]:
alphas = [0.05, 0.1, 0.5, 1,1.5, 2]
res = {}
yes_res = []
for a in alphas:
    res[a] = []
for i in range(5):
    random.shuffle(index_list)
    train_set, test_set = set(index_list[:int(train_ratio*len(raw_training))]), set(index_list[int(train_ratio*len(raw_training)):])
    train_x, train_y, valid_x, valid_y = generate_training_set(raw_training, sid, train_set, test_set, last_days)
    yes_res.append(sum([1 for i in range(len(valid_y)) if valid_y[i] > 0 ])/len(valid_y))
    for a in alphas:
        mlp = MLPC(activation = 'relu', solver = 'adam', alpha = a, hidden_layer_sizes = tuple((200,)*3), tol = 0.0001,max_iter= 200000, verbose =False, learning_rate = 'adaptive' )
        mlp.fit(train_x, train_y)
        res[a].append(accuracy(mlp, valid_x, valid_y))
print('always say yes', np.average(yes_res))
for a in alphas:
    print(a)
    print(np.average(res[a]))

always say yes 0.5410658307210031
0.05
0.4915360501567398
0.1
0.509090909090909
0.5
0.5285266457680251
1
0.5335423197492164
1.5
0.5316614420062696
2
0.5410658307210031


In [154]:
neurons = [100, 200, 300, 500, 1000]
res = {}
yes_res = []
for n in neurons:
    res[n] = []
for i in range(5):
    random.shuffle(index_list)
    train_set, test_set = set(index_list[:int(train_ratio*len(raw_training))]), set(index_list[int(train_ratio*len(raw_training)):])
    train_x, train_y, valid_x, valid_y = generate_training_set(raw_training, sid, train_set, test_set, last_days)
    yes_res.append(sum([1 for i in range(len(valid_y)) if valid_y[i] > 0 ])/len(valid_y))
    for n in neurons:
        mlp = MLPC(activation = 'relu', solver = 'adam', alpha = 0.1, hidden_layer_sizes = tuple((n,)*3), tol = 0.0001,max_iter= 200000, verbose =False, learning_rate = 'adaptive' )
        mlp.fit(train_x, train_y)
        res[n].append(accuracy(mlp, valid_x, valid_y))
print('always say yes', np.average(yes_res))
for n in neurons:
    print(n)
    print(np.average(res[n]))

always say yes 0.5391849529780564
100
0.4921630094043887
200
0.5228840125391849
300
0.5153605015673981
500
0.5153605015673981
1000
0.5191222570532915


In [155]:
layers = [1,2,3,4,5]
res = {}
yes_res = []
for l in layers:
    res[l] = []
for i in range(5):
    random.shuffle(index_list)
    train_set, test_set = set(index_list[:int(train_ratio*len(raw_training))]), set(index_list[int(train_ratio*len(raw_training)):])
    train_x, train_y, valid_x, valid_y = generate_training_set(raw_training, sid, train_set, test_set, last_days)
    yes_res.append(sum([1 for i in range(len(valid_y)) if valid_y[i] > 0 ])/len(valid_y))
    for l in layers:
        mlp = MLPC(activation = 'relu', solver = 'adam', alpha = 1, hidden_layer_sizes = tuple((200,)*l), tol = 0.0001,max_iter= 200000, verbose =False, learning_rate = 'adaptive' )
        mlp.fit(train_x, train_y)
        res[l].append(accuracy(mlp, valid_x, valid_y))
print('always say yes', np.average(yes_res))
for l in layers:    
    print(l)
    print(np.average(res[l]))

always say yes 0.5398119122257053
1
0.503448275862069
2
0.5059561128526646
3
0.5047021943573669
4
0.5216300940438872
5
0.5398119122257053


In [189]:
random.shuffle(index_list)
train_set, test_set = set(index_list[:int(train_ratio*len(raw_training))]), set(index_list[int(train_ratio*len(raw_training)):])
train_x, train_y, valid_x, valid_y = generate_training_set(raw_training, sid, train_set, test_set, last_days)
sum([1 for i in range(len(valid_y)) if valid_y[i] > 0 ])/len(valid_y)

0.5360501567398119

In [204]:
mlp = MLPC(activation = 'relu', solver = 'adam', alpha = 0.1, hidden_layer_sizes = tuple((200,)*3), tol = 0.0001,max_iter= 200000, verbose =False, learning_rate = 'adaptive' )
mlp.fit(train_x, train_y)
accuracy(mlp, valid_x, valid_y)

0.5454545454545454

In [215]:
new_X, new_y = pd.concat([train_x,valid_x]).reset_index(drop = True),  train_y + valid_y

In [217]:
test_x = generate_test_set(raw_test, sid, last_days)

In [218]:
mlp = MLPC(activation = 'relu', solver = 'adam', alpha = 0.1, hidden_layer_sizes = tuple((200,)*3), tol = 0.0001,max_iter= 200000, verbose =False, learning_rate = 'adaptive' )
mlp.fit(new_X, new_y)


MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 200, 200), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=200000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [219]:
final_result = mlp.predict(test_x)

In [223]:
sub_data['stock_directionality'] = final_result

In [226]:
sub_data.drop('index', axis = 1, inplace = True)

In [227]:
sub_data.to_csv(write_dic, index = False)